In [195]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import time

In [196]:
torch.cuda.is_available()

True

In [197]:
!pip install transformers huggingface_hub
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [198]:
!pip install datasets

In [199]:
from datasets import Dataset, load_dataset, DatasetDict

In [200]:
data = pd.read_csv('Random_2000.csv')

In [201]:
data_annotated = data[:1112]

In [202]:
type(data_annotated)

pandas.core.frame.DataFrame

In [203]:
train_data = data_annotated[:556]
test_data = data_annotated[556:]

In [204]:
type(train_data)

pandas.core.frame.DataFrame

In [205]:
train_data.to_csv('train_data_2000.csv', index=False)
time.sleep(30)

In [206]:
test_data.to_csv('test_data_2000.csv', index=False)

In [207]:
data_files = {'train': 'train_data_2000.csv', 'test': 'test_data_2000.csv'}

In [208]:
random_2000_dataset = load_dataset('csv', data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [209]:
random_2000_dataset

DatasetDict({
    train: Dataset({
        features: ['Comment', 'SentimentalLabel'],
        num_rows: 562
    })
    test: Dataset({
        features: ['Comment', 'SentimentalLabel'],
        num_rows: 559
    })
})

In [210]:
random_2000_dataset = random_2000_dataset.filter(lambda x: x['Comment'] is not None)

Filter:   0%|          | 0/562 [00:00<?, ? examples/s]

Filter:   0%|          | 0/559 [00:00<?, ? examples/s]

In [211]:
random_2000_dataset = random_2000_dataset.filter(lambda x: x['SentimentalLabel'] is not None)

Filter:   0%|          | 0/560 [00:00<?, ? examples/s]

Filter:   0%|          | 0/553 [00:00<?, ? examples/s]

In [212]:
random_2000_dataset

DatasetDict({
    train: Dataset({
        features: ['Comment', 'SentimentalLabel'],
        num_rows: 554
    })
    test: Dataset({
        features: ['Comment', 'SentimentalLabel'],
        num_rows: 548
    })
})

In [214]:
small_train_dataset = random_2000_dataset["train"].shuffle(seed=42).select([i for i in list(range(500))])
small_test_dataset = random_2000_dataset["test"].shuffle(seed=42).select([i for i in list(range(500))])
print(small_train_dataset[0])
print(small_test_dataset[0])

{'Comment': 'hi bro can you recreate the inventions of nikola tesla whose words are being hidden by america or someone for their own sake and as far as i know i39ve read that there are also inventions of nikola tesla such as wireless charging like the example you gave i hope in this world there are people who can make new discoveries or restore old inventions that may be really needed in this day and age to save electricity', 'SentimentalLabel': 'Pos'}
{'Comment': 'as i understand it combustion vehicles generally have an energy efficiency of 20 let s consider the opportunity costs even if we only went with the 40 efficient charging system and even if only half the extra grid energy were to come from clean sources and even if none of the vehicles had a solar panel roof an inductively charged road network would still yield a net reduction in emissionsbonuses  what emissions continued to happen would do so in large remote static facilities that can have massive filters as opposed to gridl

In [216]:
len(small_test_dataset)

500

In [217]:
small_train_dataset[:5]

{'Comment': ['hi bro can you recreate the inventions of nikola tesla whose words are being hidden by america or someone for their own sake and as far as i know i39ve read that there are also inventions of nikola tesla such as wireless charging like the example you gave i hope in this world there are people who can make new discoveries or restore old inventions that may be really needed in this day and age to save electricity',
  'check this out',
  'the man who pioneerd 21st century with others like einstein father of ac and lasersome dude with 3 braincells quothe didnt understand his devicequot',
  'quottesla did not uderstud his invetions  quot are u mad  this tesla coil is not right tesla coil where are longitudual waves in you constraction it is wrong that u thing that is original tesla coil   stupid man',
  'tesla absolutely knew what he was doing people like thomas edison and j p morgan worked on destroying tesla credibility however his inventions will be distribute all through o

In [218]:
type(small_train_dataset)

datasets.arrow_dataset.Dataset

In [219]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [220]:
def preprocess_function(examples):
  return tokenizer(examples['Comment'], truncation=True)

In [221]:
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]